In [116]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import seaborn as sns


from sklearn.grid_search import GridSearchCV


from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier,\
                             AdaBoostClassifier, GradientBoostingClassifier
import xgboost
from sklearn.svm import SVC

from sklearn.metrics import confusion_matrix, classification_report, roc_curve, log_loss
from sklearn.model_selection import cross_val_score

######################################  Preprocessing  ######################################
from sklearn.preprocessing import binarize, label_binarize
from skimage.transform import rescale
from scipy import ndimage as ndi

######################################  Image contol  ######################################

import cv2
from IPython.core.display import Image, display

######################################       PCA      ######################################
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA

import time

In [50]:
# encogin categorical values for further processing
def encode_cat(df):
    """
       Input : train_df 
       Output : encoded_train_df
    """
    # get categorical(type='object') variable
    cat_var_df = df.select_dtypes(include=['object']).copy()    
    
    cleanup_nums = {}
    
    # species columns encoding
    spe_index = 0
    spe_enc_dict = {}
    for key in set(cat_var_df['species'].values):
        spe_enc_dict[key] = spe_index
        spe_index += 1
    cleanup_nums['species'] = spe_enc_dict
    
    try:
        cat_var_df.replace(cleanup_nums, inplace=True)
    except:
        pass
    
    df = df.drop(cat_var_df.columns, axis=1)
    df = pd.concat([df, cat_var_df], axis=1)
    return df

In [52]:
dfX_std_scaled_3area_pca_df = pd.read_csv('../dfX_std_scaled_3area_pca_df_20171116.csv')
dfy = encode_cat(pd.read_csv('../train.csv'))['species']

In [4]:
print(dfX_std_scaled_3area_pca_df.shape)
print(dfy.shape)

(990, 245)
(990,)


# Model setting 1

In [26]:
# models abbreviate 1
qda = QuadraticDiscriminantAnalysis()
mnb = MultinomialNB()
mnb_al2 = MultinomialNB(alpha=2.0)
gnb = GaussianNB()
lg = LogisticRegression()
lg_lbf_mul = LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial')
dt_d10 = DecisionTreeClassifier(criterion='entropy', max_depth=10)
dt_d20 = DecisionTreeClassifier(criterion='entropy', max_depth=20)
vc_lr_gnb = VotingClassifier(estimators=[('lr',  LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial')), \
                                         ('gnb', GaussianNB())], voting='soft', weights=[1, 1])
bc_vc_lr_qda_gnb = BaggingClassifier(base_estimator=vc_lr_gnb, n_estimators=100, \
                                  bootstrap=True, bootstrap_features=True)
rc_d20_e1000 = RandomForestClassifier(max_depth=20, n_estimators=1000)
etc_m10_e500 = ExtraTreesClassifier(max_depth=10, n_estimators=500)
ab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5, random_state=0), 
                               algorithm="SAMME", n_estimators=100)
gc_d10_e100 = GradientBoostingClassifier(n_estimators=100, max_depth=10)
xb = xgboost.XGBClassifier(n_estimators=100, max_depth=10)
pt_iter500 = Perceptron(max_iter=500, eta0=0.1)
sgdc_ll_iter1000 = SGDClassifier(loss="log", max_iter=1000)
svc_c3_4_lin = SVC(C=0.75, kernel='linear')


In [22]:
# x should be positive : ['qda', qda],['mnb', mnb], ['mnb_al2', mnb_al2],
    
model_lst1 = [    
    ['gnb', gnb],
    ['lg', lg],
    ['lg_lbf_mul', lg_lbf_mul],
    ['dt_d10', dt_d10],
    ['dt_d20', dt_d20],
    ['vc_lr_gnb', vc_lr_gnb],
    ['bc_vc_lr_qda_gnb', bc_vc_lr_qda_gnb],
    ['rc_d20_e1000', rc_d20_e1000],
    ['etc_m10_e500', etc_m10_e500],
    ['ab', ab],
    ['gc_d10_e100', gc_d10_e100 ],
    ['xb', xb],
    ['pt_iter500', pt_iter500],
    ['sgdc_ll_iter1000', sgdc_ll_iter1000],
    ['svc_c3_4_lin', svc_c3_4_lin]    
]

In [23]:
for model in model_lst1:
    print('='*50)
    print('Model : ', model[0])
    print(cross_val_score(model[1],dfX_std_scaled_3area_pca_df, dfy, scoring='neg_log_loss', cv=5))

Model :  gnb
[-13.43200316 -14.69802855 -17.30904952 -15.35056729 -16.39719687]
Model :  lg
[-0.65832899 -0.61102704 -0.64260403 -0.63985792 -0.63484268]
Model :  lg_lbf_mul
[-0.12131882 -0.09936317 -0.13482313 -0.1167943  -0.09830722]
Model :  dt_d10
[-12.21067852 -11.86180199 -12.21067852 -11.16404894 -12.21067852]
Model :  dt_d20
[ -9.24522802 -11.16404894 -10.64073414 -12.21067852 -11.3384872 ]
Model :  vc_lr_gnb
[-0.35054527 -0.36602723 -0.45865696 -0.40302072 -0.40387894]
Model :  bc_vc_lr_qda_gnb


/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:233: RuntimeWarning: invalid value encountered in true_divide
  new_mu = np.average(X, axis=0, weights=sample_weight / n_new)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:235: RuntimeWarning: invalid value encountered in true_divide
  weights=sample_weight / n_new)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:233: RuntimeWarning: invalid value encountered in true_divide
  new_mu = np.average(X, axis=0, weights=sample_weight / n_new)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:235: RuntimeWarning: invalid value encountered in true_divide
  weights=sample_weight / n_new)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/naive_bayes.py:431: RuntimeWarning: divide by zero encountered in log
  jointi = np.log(self.class_prior_[i])
/home/dockeruser/anaconda3/lib/python3.6/site-packages/scipy/special/_log

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [24]:
# infinity error : ['bc_vc_lr_qda_gnb', bc_vc_lr_qda_gnb],

model_lst2 = [        
    ['rc_d20_e1000', rc_d20_e1000],
    ['etc_m10_e500', etc_m10_e500],
    ['ab', ab],
    ['gc_d10_e100', gc_d10_e100 ],
    ['xb', xb],
    ['pt_iter500', pt_iter500],
    ['sgdc_ll_iter1000', sgdc_ll_iter1000],
    ['svc_c3_4_lin', svc_c3_4_lin]    
]

In [25]:
for model in model_lst2:
    print('='*50)
    print('Model : ', model[0])
    print(cross_val_score(model[1],dfX_std_scaled_3area_pca_df, dfy, scoring='neg_log_loss', cv=5))

Model :  rc_d20_e1000
[-0.86160614 -0.87034713 -0.85490459 -0.81952618 -0.83339129]
Model :  etc_m10_e500
[-1.11860379 -1.14237064 -1.11640589 -1.10306674 -1.10211474]
Model :  ab
[-4.59455992 -4.59460722 -4.59458221 -4.59457786 -4.59455931]
Model :  gc_d10_e100
[-2.70765633 -2.33461041 -2.99777586 -3.03939294 -2.72905026]
Model :  xb


TypeError: estimator should be an estimator implementing 'fit' method, <module 'xgboost' from '/home/dockeruser/anaconda3/lib/python3.6/site-packages/xgboost/__init__.py'> was passed

In [27]:
# 

model_lst3 = [  
    ['xb', xb],
    ['pt_iter500', pt_iter500],
    ['sgdc_ll_iter1000', sgdc_ll_iter1000],
    ['svc_c3_4_lin', svc_c3_4_lin]    
]

In [28]:
for model in model_lst3:
    print('='*50)
    print('Model : ', model[0])
    print(cross_val_score(model[1],dfX_std_scaled_3area_pca_df, dfy, scoring='neg_log_loss', cv=5))

Model :  xb
[-0.8168922  -0.74283495 -0.79232421 -0.82844062 -0.72984968]
Model :  pt_iter500


AttributeError: 'Perceptron' object has no attribute 'predict_proba'

In [29]:
# 


model_lst4 = [  
    ['sgdc_ll_iter1000', sgdc_ll_iter1000],
    ['svc_c3_4_lin', svc_c3_4_lin]    
]

In [30]:
for model in model_lst4:
    print('='*50)
    print('Model : ', model[0])
    print(cross_val_score(model[1],dfX_std_scaled_3area_pca_df, dfy, scoring='neg_log_loss', cv=5))

Model :  sgdc_ll_iter1000


/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)
/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


[-1.29365224 -0.94288094 -1.50288058 -1.78090544 -1.0959655 ]
Model :  svc_c3_4_lin


AttributeError: predict_proba is not available when  probability=False

In [84]:
cross_val_score(SVC(C=1.5, kernel='linear'),\
         dfX_std_scaled_3area_pca_df, dfy, scoring='log_loss', cv=5)

/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


AttributeError: predict_proba is not available when  probability=False

In [99]:
model_svc = SVC(C=0.1, kernel='rbf').fit(dfX_std_scaled_3area_pca_df, dfy)
dfy_svc_pred = model_svc.decision_function(dfX_std_scaled_3area_pca_df)

In [100]:
dfy_svc_pred_proba = dfy_svc_pred/sum(dfy_svc_pred[0])

In [101]:
dfy_onehot = label_binarize(dfy, classes=range(99))

In [63]:
log_loss(dfy, dfy_svc_pred)

ValueError: y_true and y_pred contain different number of classes 99, 2. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49
 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74
 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98]

In [62]:
len(np.unique(dfy_svc_pred))

99

In [43]:
type(dfy)

pandas.core.series.Series

In [45]:
cross_val_score(lg_lbf_mul, dfX_std_scaled_3area_pca_df, dfy, scoring='accuracy', cv=5)

array([ 0.98484848,  0.98989899,  0.97979798,  0.98989899,  0.99494949])

In [113]:
model_lst5 = [    
    ['gnb', gnb],
    ['lg', lg],
    ['lg_lbf_mul', lg_lbf_mul],
    ['dt_d10', dt_d10],
    ['dt_d20', dt_d20],
    ['vc_lr_gnb', vc_lr_gnb],    
    ['rc_d20_e1000', rc_d20_e1000],
    ['etc_m10_e500', etc_m10_e500],
    ['ab', ab],
    ['gc_d10_e100', gc_d10_e100 ],
    ['xb', xb],
    ['pt_iter500', pt_iter500],
    ['sgdc_ll_iter1000', sgdc_ll_iter1000],
    ['svc_c3_4_lin', svc_c3_4_lin]    
]

In [114]:
for model in model_lst5:
    print('='*50)
    print('Model : ', model[0])
    print(cross_val_score(model[1],dfX_std_scaled_3area_pca_df, dfy, scoring='accuracy', cv=5))

Model :  gnb
[ 0.61111111  0.57070707  0.49494949  0.55555556  0.52525253]
Model :  lg
[ 0.97979798  0.98989899  0.97474747  0.97474747  0.97474747]
Model :  lg_lbf_mul
[ 0.98484848  0.98989899  0.97979798  0.98989899  0.99494949]
Model :  dt_d10
[ 0.65656566  0.64646465  0.65151515  0.68686869  0.65656566]
Model :  dt_d20
[ 0.67676768  0.7020202   0.65151515  0.64141414  0.65656566]
Model :  vc_lr_gnb
[ 0.61616162  0.57070707  0.49494949  0.55555556  0.53535354]
Model :  rc_d20_e1000
[ 0.97474747  0.97979798  0.98484848  0.97979798  0.98484848]
Model :  etc_m10_e500
[ 0.97474747  0.96969697  0.96464646  0.97979798  0.98484848]
Model :  ab
[ 0.54545455  0.39393939  0.38383838  0.58080808  0.4040404 ]
Model :  gc_d10_e100
[ 0.59090909  0.67676768  0.61111111  0.6010101   0.63636364]
Model :  xb
[ 0.86868687  0.9040404   0.87373737  0.87373737  0.86868687]
Model :  pt_iter500
[ 0.91919192  0.93939394  0.87878788  0.91414141  0.91414141]
Model :  sgdc_ll_iter1000
[ 0.93434343  0.93939394 

In [ ]:
lg_lbf_mul
rc_d20_e1000
 etc_m10_e500
    svc_c3_4_lin

# Model setting 2 : improving individual model

In [117]:
# models abbreviate 2
gnb = GaussianNB()
lg = LogisticRegression()
lg_lbf_mul = LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial')


rc_d20_e1000 = RandomForestClassifier(max_depth=20, n_estimators=1000)
rc_d20_ent_e1000 = RandomForestClassifier(max_depth=20, criterion='entropy', n_estimators=1000)
etc_m10_e500 = ExtraTreesClassifier(max_depth=10, n_estimators=500)

svc_c3_4_lin = SVC(C=0.75, kernel='linear')
svc_c3_4_rbf_ga1 = SVC(C=0.75, kernel='rbf', gamma=1)
svc_c3_4_rbf_ga2 = SVC(C=0.75, kernel='rbf', gamma=2)

############################################## mixed ##############################################
vc_lr_gnb = VotingClassifier(estimators=[('lr',  LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial')), \
                                         ('gnb', GaussianNB())], voting='soft', weights=[1, 1])


############################################## option ##############################################
ab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5, random_state=0), 
                               algorithm="SAMME", n_estimators=100)
gc_d10_e100 = GradientBoostingClassifier(n_estimators=100, max_depth=10)
xb = xgboost.XGBClassifier(n_estimators=100, max_depth=10)


### target : feedback from model 5
- lg_lbf_mul
- rc_d20_e1000
- etc_m10_e500
- svc_c3_4_lin
<pre>
Model :  lg_lbf_mul
[ 0.98484848  0.98989899  0.97979798  0.98989899  0.99494949]
==================================================
Model :  rc_d20_e1000
[ 0.97474747  0.97979798  0.98484848  0.97979798  0.98484848]
==================================================
Model :  etc_m10_e500
[ 0.97474747  0.96969697  0.96464646  0.97979798  0.98484848]
==================================================
Model :  svc_c3_4_lin
[ 0.98989899  0.98989899  0.97979798  0.99494949  1.        ]
</pre>

In [115]:
model_lst1 = [    
    ['gnb', gnb],
    ['lg', lg],
    ['lg_lbf_mul', lg_lbf_mul],
    ['dt_d10', dt_d10],
    ['dt_d20', dt_d20],
    ['vc_lr_gnb', vc_lr_gnb],
    ['bc_vc_lr_qda_gnb', bc_vc_lr_qda_gnb],
    ['rc_d20_e1000', rc_d20_e1000],
    ['etc_m10_e500', etc_m10_e500],
    ['ab', ab],
    ['gc_d10_e100', gc_d10_e100 ],
    ['xb', xb],
    ['pt_iter500', pt_iter500],
    ['sgdc_ll_iter1000', sgdc_ll_iter1000],
    ['svc_c3_4_lin', svc_c3_4_lin]    
]

## Logistic Regeression

In [119]:
c_range = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
tol_range = [0.01, 0.001, 0.0005, 0.0001, 0.00005, 0.00001]
param_grid = [
    {'C': c_range, 'tol': tol_range}]

gs = GridSearchCV(estimator=lg_lbf_mul, param_grid=param_grid, scoring='neg_log_loss', cv=10, n_jobs=1)
%time gs = gs.fit(dfX_std_scaled_3area_pca_df, dfy)

CPU times: user 22min 11s, sys: 5min 24s, total: 27min 35s
Wall time: 14min 2s


In [121]:
gs.best_params_

{'C': 1000.0, 'tol': 0.0005}

In [122]:
gs.best_score_

-0.02830373197194751

In [124]:
gs.best_estimator_

LogisticRegression(C=1000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='multinomial', n_jobs=1, penalty='l2',
          random_state=None, solver='lbfgs', tol=0.0005, verbose=0,
          warm_start=False)

## Random Forest 